# Métodos de pré-processamento de dados com Scikit-Learn - Parte 2

Às vezes pode ser difícil encontrar o caminho pela floresta de técnicas de pré-processamento. Sklearn sua biblioteca de pré-processamento forma uma base sólida para orientá-lo nesta importante tarefa no pipeline de ciência de dados. Embora o Sklearn tenha uma documentação bastante sólida, muitas vezes ele perde a agilidade e a intuição entre os diferentes conceitos.

Serão tratados os seguintes assuntos:<br>
•	Missing values<br>
•	Polynomial features<br>
•	Categorical features<br>
•	Numerical features<br>
•	Custom transformations<br>
•	Feature scaling<br>
•	Normalization<br>
Observe que as etapas três e quatro podem ser executadas de forma intercambiável, pois essas transformações devem ser executadas independentemente umas das outras.

#### 1. Missing values

A manipulação de valores ausentes é uma tarefa essencial de pré-processamento que pode deteriorar drasticamente seu modelo quando não realizada com cuidado suficiente. Algumas perguntas devem surgir ao lidar com valores ausentes:
Tenho valores ausentes? Como eles são expressos nos dados? Devo reter amostras com valores ausentes? Ou devo substituí-los? Em caso afirmativo, por quais valores eles devem ser substituídos?<br>
Antes de começar a lidar com valores ausentes, é importante identificar os valores ausentes e saber com qual valor eles são substituídos. Você deve conseguir descobrir isso combinando as informações de metadados com a análise exploratória.<br>
Depois de saber um pouco mais sobre os dados ausentes, você deve decidir se deseja ou não manter as entradas com dados ausentes. De acordo com Chris Albon (Machine Learning with Python Cookbook), essa decisão deve depender parcialmente de quão aleatórios são os valores ausentes.<br>
Se forem completamente aleatórios, não fornecem nenhuma informação extra e podem ser omitidos. Por outro lado, se eles não forem aleatórios, o fato de um valor estar faltando é uma informação e pode ser expresso como um recurso binário extra.<br>
Lembrar também de que excluir uma observação inteira porque tem um valor ausente pode ser uma má decisão e levar à perda de informações. Assim como manter uma linha inteira de valores ausentes porque tem um valor ausente significativo, pode não ser sua melhor opção.<br>
Vamos materializar essa teoria com alguns exemplos de codificação usando o MissingIndicator do sklearn. Para dar algum significado ao nosso código, criaremos um conjunto de dados muito pequeno com três recursos e cinco amostras. Os dados contêm valores ausentes óbvios expressos como não um número ou 999.

In [12]:
import numpy as np
import pandas as pd

X = pd.DataFrame(
    np.array([5,7,8, np.NaN, np.NaN, np.NaN, -5,
              0,25,999,1,-1, np.NaN, 0, np.NaN])\
              .reshape((5,3)))
X.columns = ['f1', 'f2', 'f3'] 
#feature 1, feature 2, feature 3
X

,f1,f2,f3
0,5.0,7.0,8.0
1,NaN,NaN,NaN
2,-5.0,0.0,25.0
3,999.0,1.0,-1.0
4,NaN,0.0,NaN


Conjunto de dados com três recursos e cinco amostras<br><br>
Dê uma olhada rápida nos dados para saber onde estão situados os valores ausentes. Linhas ou colunas com muitos valores ausentes não significativos podem ser excluídas dos seus dados com a função dropna dos pandas. Vamos olhar nos parâmetros mais importantes:<br>
• eixo: 0 para linhas, 1 para colunas<br>
• tresh: o número de não-NaNs para não descartar uma linha ou coluna<br>
• inplace: atualize o quadro<br>
Atualizamos nosso conjunto de dados excluindo todas as linhas (eixo=0) apenas com valores ausentes. Observe que, neste caso, em vez de definir tresh como 1, você também pode definir o parâmetro how como 'all'. Como resultado, nossa segunda amostra é descartada, pois consiste apenas em valores ausentes. Observe que redefinimos o índice e descartamos a coluna de índice antiga para conveniência futura.

In [13]:
X.dropna(axis=0, thresh=1, inplace=True)
X.reset_index(inplace=True)
X.drop(['index'], axis=1, inplace=True)
X

,f1,f2,f3
0,5.0,7.0,8.0
1,-5.0,0.0,25.0
2,999.0,1.0,-1.0
3,NaN,0.0,NaN


Vamos também criar alguns recursos booleanos extras que nos informam se uma amostra tem um valor ausente para um determinado recurso. Comece importando o MissingIndicator de sklearn.impute (observe que a versão 0.20.0 é necessária (atualize com 'conda update scikit-learn')).<br>
Infelizmente, o MissingIndicator não oferece suporte a vários tipos de valores ausentes (consulte esta pergunta no Stackoverflow). Por isso, temos que converter os 999 valores em nosso dataframe para NaN's. Em seguida, criamos, ajustamos e transformamos um objeto MissingIndicator que detectará todos os NaNs em nossos dados.<br>
A partir deste indicador, podemos criar um novo dataframe com valores booleanos indicando se uma instância possui um valor ausente para um determinado feature. Mas por que temos apenas duas novas colunas enquanto tínhamos três features originais? Após a exclusão de nossa segunda amostra, f2 não tinha mais valores ausentes. Se o MissingIndicator não detectar nenhum valor ausente em um recurso, ele não criará um novo feature desse feature.<br>
Adicionaremos esses novos recursos posteriormente aos nossos dados originais, por enquanto podemos armazená-los na variável indicadora.

In [14]:
from sklearn.impute import MissingIndicator

X.replace({999.0 : np.NaN}, inplace=True)
print(X)
indicator = MissingIndicator(missing_values=np.NaN)
print(indicator)
indicator = indicator.fit_transform(X)
print(indicator)
indicator = pd.DataFrame(indicator, columns=['m1', 'm3'])
indicator, X

    f1   f2    f3
0  5.0  7.0   8.0
1 -5.0  0.0  25.0
2  NaN  1.0  -1.0
3  NaN  0.0   NaN
MissingIndicator()
[[False False]
 [False False]
 [ True False]
 [ True  True]]


(      m1     m3
 0  False  False
 1  False  False
 2   True  False
 3   True   True,
     f1   f2    f3
 0  5.0  7.0   8.0
 1 -5.0  0.0  25.0
 2  NaN  1.0  -1.0
 3  NaN  0.0   NaN)

Depois de decidir manter (alguns de) seus valores ausentes e criar indicadores de valor ausente, a próxima pergunta é se deve substituir os valores ausentes. A maioria dos algoritmos de aprendizagem tem um desempenho ruim quando os valores ausentes são expressos como não um número (np.NaN) e precisam de alguma forma de imputação de valores ausentes. Esteja ciente de que algumas bibliotecas e algoritmos, como o XGBoost, podem manipular valores ausentes e imputar esses valores automaticamente por aprendizagem.

#### 2. Input values

Para preencher os valores ausentes com estratégias comuns, o sklearn fornece um SimpleImputer. As quatro estratégias principais são média, most_frequent, mediana e constante (não se esqueça de definir o parâmetro fill_value). No exemplo abaixo, imputamos valores ausentes para nosso dataframe X com a média do feature.

In [15]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit_transform(X)
imp

SimpleImputer()

Observe que os valores retornados são colocados em um array Numpy e perdemos todas as meta-informações. Como todas essas estratégias podem ser imitadas em pandas, usaremos o método pandas fillna para imputar valores ausentes. Para 'mean', podemos usar o seguinte código. Essa implementação de pandas também oferece opções para preencher para frente (ffill) ou para trás (bfill), que são convenientes ao trabalhar com séries temporais.

In [16]:
X.fillna(X.mean(), inplace=True)
X

,f1,f2,f3
0,5.0,7.0,8.000000
1,-5.0,0.0,25.000000
2,0.0,1.0,-1.000000
3,0.0,0.0,10.666667


In [17]:
X1=X

Outras formas populares de imputar dados ausentes são agrupar os dados com o algoritmo k-nearest neighbor (KNN) ou interpolar os valores usando uma ampla variedade de métodos de interpolação. Ambas as técnicas não são implementadas na biblioteca de pré-processamento do sklearn e não serão discutidas aqui.

#### 3. Polynomial features

A criação de recursos polinomiais é uma maneira simples e comum de engenharia de recursos que adiciona complexidade aos dados numéricos de entrada combinando recursos.<br>
Os features polinomiais são frequentemente criados quando queremos incluir a noção de que existe uma relação não linear entre os features e o destino. Eles são usados ​​principalmente para adicionar complexidade a modelos lineares com poucos features, ou quando suspeitamos que o efeito de um feature é dependente em outro feature.<br>
Antes de lidar com valores ausentes, precisa decidir se deseja usar features polinomiais ou não. Se por exemplo, substituir todos os valores ausentes por 0, todos os produtos cruzados usando esse feature serão 0. Além disso, se você não substituir os valores ausentes (NaN), a criação de features polinomiais gerará um erro de valor na fase fit_transform , pois a entrada deve ser finita.<br>
A este respeito, substituir os valores omissos pela mediana ou pela média parece ser uma escolha razoável. Como não tenho certeza absoluta sobre isso e não consigo encontrar informações consistentes, fiz esta pergunta no StackExchange de ciência de dados.<br>
O Sklearn fornece uma classe PolynomialFeatures para criar features polinomiais do zero. O parâmetro de grau determina o grau máximo do polinómio. Por exemplo, quando o grau é definido como dois e X=x1, x2, os recursos criados serão 1, x1, x2, x1², x1x2 e x2². O parâmetro transaction_only informa à função que queremos apenas os recursos de interação, ou seja, 1, x1, x2 e x1x2.<br>
Aqui, criamos recursos polinomiais até o terceiro grau e único recurso de interação. Como resultado temos quatro novas features: f1.f2, f1.f3, f2.f3 e f1.f2.f3. Observe que nossos features originais também estão incluídos na saída e separamos os novos recursos para adicionar aos nossos dados posteriormente.

In [18]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3, interaction_only=True)
polynomials = pd.DataFrame(poly\
                           .fit_transform(X), 
                           columns=['0','1','2','3', 
                                    'p1', 'p2', 'p3', 'p4'])\
                                        [['p1', 'p2', 'p3', 'p4']]
polynomials

,p1,p2,p3,p4
0,35.0,40.0,56.0,280.0
1,-0.0,-125.0,0.0,-0.0
2,0.0,-0.0,-1.0,-0.0
3,0.0,0.0,0.0,0.0


Assim como com qualquer outra forma de engenharia de features, é importante criar features polinomiais antes de fazer qualquer dimensionamento de features.
Agora, vamos concatenar nossos novos features de indicadores ausentes e features polinomiais para nossos dados com o método pandas concat.

In [19]:
X = pd.concat([X, indicator, polynomials], axis=1)
X

,f1,f2,f3,m1,m3,p1,p2,p3,p4
0,5.0,7.0,8.000000,False,False,35.0,40.0,56.0,280.0
1,-5.0,0.0,25.000000,False,False,-0.0,-125.0,0.0,-0.0
2,0.0,1.0,-1.000000,True,False,0.0,-0.0,-1.0,-0.0
3,0.0,0.0,10.666667,True,True,0.0,0.0,0.0,0.0


Dataframe com features originais (f), indicadores de valor ausente (m) e features polinomiais (p)

#### 4. Categorical features

A manipulação de dados categóricos é outro processo essencial durante o pré-processamento de dados. Infelizmente, a biblioteca de machine learning do sklearn não suporta o tratamento de dados categóricos. Mesmo para modelos baseados em árvore, é necessário converter recursos categóricos em uma representação numérica.<br>
Antes de começar a transformar seus dados, é importante descobrir se o feature em que está a trabalhar é ordinal (em oposição a nominal). Uma feição ordinal é melhor descrita como uma feição com categorias naturais e ordenadas e as distâncias entre as categorias não são conhecidas.<br>
Depois de saber em que tipo de dados categóricos você está trabalhando, pode escolher uma ferramenta de transformação adequada. No sklearn, será um OrdinalEncoder para dados ordinais e um OneHotEncoder para dados nominais.
Vamos considerar um exemplo simples para demonstrar como ambas as classes estão a funcionar. Crie um dataframe com cinco entradas e três características: sexo, tipo sanguíneo e nível de escolaridade.

In [21]:
X = pd.DataFrame(
    np.array(['M', 'O-', 'medium',
             'M', 'O-', 'high',
              'F', 'O+', 'high',
              'F', 'AB', 'low',
              'F', 'B+', np.NaN])
              .reshape((5,3)))
X.columns = ['sex', 'blood_type', 'edu_level']
X

,sex,blood_type,edu_level
0,M,O-,medium
1,M,O-,high
2,F,O+,high
3,F,AB,low
4,F,B+,nan


Se observar o dataframe, deve notar que o nível de escolaridade é o único recurso ordinal (pode ser ordenado e a distância entre as categorias não é conhecida). Começaremos codificando esse recurso com a classe OrdinalEncoder. Importe a classe e crie uma nova instância. Em seguida, atualize o feature de nível educacional ajustando e transformando o feature no codificador. O resultado deve ficar como abaixo.

In [22]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
X.edu_level = encoder.fit_transform(X.edu_level.values.reshape(-1, 1))
X

,sex,blood_type,edu_level
0,M,O-,2.0
1,M,O-,0.0
2,F,O+,0.0
3,F,AB,1.0
4,F,B+,3.0


Observe que temos um problema bastante irritante aqui: o nosso valor ausente é codificado como uma classe separada (3.0). Uma análise detalhada da documentação revela que ainda não há solução para esse problema. Um bom sinal é que os programadores do sklearn estão a discutir as possibilidades de implementar uma solução adequada.<br>
Outro problema é que a ordem dos nossos dados não é respeitada. Felizmente, isso pode ser resolvido passando uma lista ordenada de valores exclusivos para o feature, e para o parâmetro de categorias.<br>
codificador = OrdinalEncoder(categories=['baixo', 'médio', 'alto'])
Para resolver o primeiro problema, temos que recorrer aos pandas. O método factorize fornece uma alternativa que pode lidar com valores ausentes e respeita a ordem de nossos valores. A primeira etapa é converter o recurso em um pandas Categorical ordenado. Passe uma lista de categorias (incluindo uma categoria para valores ausentes) e defina o parâmetro ordenado como True.

In [27]:
cat = pd.Categorical(X.edu_level, 
                     categories=['missing', 'low', 'medium', 'high'], 
                     ordered=True)
cat

[NaN, NaN, NaN, NaN, NaN]
Categories (4, object): ['missing' < 'low' < 'medium' < 'high']

Substitua os valores ausentes pela categoria ausente.

In [28]:
cat.fillna('missing')
cat

[NaN, NaN, NaN, NaN, NaN]
Categories (4, object): ['missing' < 'low' < 'medium' < 'high']

Em seguida, fatorize o Categorical com o parâmetro de classificação definido como Verdadeiro e atribua a saída ao feature de nível de educação.

In [29]:
labels, unique = pd.factorize(cat, sort=True)
X.edu_level = labels
X

,sex,blood_type,edu_level
0,M,O-,-1
1,M,O-,-1
2,F,O+,-1
3,F,AB,-1
4,F,B+,-1


Os resultados são mais satisfatórios desta vez, pois os dados são numéricos, ainda ordenados e os valores ausentes são substituídos por 0. Observe que substituir os valores ausentes pelo menor valor pode nem sempre ser a melhor escolha. Outras opções são colocá-lo na categoria mais comum ou colocá-lo na categoria do valor no meio quando o recurso é classificado.

Vamos nos voltar para os outros dois featurea nominais agora. Lembrar que não podemos substituir esses features por um número, pois isso implicaria que os features tenham uma ordem, o que não é verdade em caso de sexo ou tipo sanguíneo.
A maneira mais popular de codificar recursos nominais é a codificação one-hot. Essencialmente, cada recurso categórico com n categorias é transformado em n features binários.<br>
Vamos dar uma olhada no nosso exemplo para deixar as coisas claras. Comece importando a classe OneHotEncoder e criando uma nova instância com o tipo de dados de saída definido como inteiro. Isso não altera em nada a forma como nossos dados serão interpretados, mas melhorará a legibilidade de nossa saída.<br>
Então, ajuste e transforme nossas duas categóricas nominais. A saída dessa transformação será uma matriz esparsa, isso significa que teremos que transformar a matriz em um array (.toarray()) antes de podermos colocá-la em um dataframe. Você pode omitir esta etapa definindo o parâmetro sparse como False ao iniciar uma nova instância de classe. Atribua nomes de coluna e a saída está pronta para ser adicionada aos outros dados (feature) edu_level).

In [31]:
from sklearn.preprocessing import OneHotEncoder

# onehot = OneHotEncoder(dtype=np.int, sparse=True)
onehot = OneHotEncoder(dtype=int, sparse_output=True)
nominals = pd.DataFrame(
    onehot.fit_transform(X[['sex', 'blood_type']]).toarray(),
    columns=['F', 'M', 'AB', 'B+','O+', 'O-'])
nominals['edu_level'] = X.edu_level
nominals

,F,M,AB,B+,O+,O-,edu_level
0,0,1,0,0,0,1,-1
1,0,1,0,0,0,1,-1
2,1,0,0,0,1,0,-1
3,1,0,1,0,0,0,-1
4,1,0,0,1,0,0,-1


![](Picture%202.png)<br>
Dados codificados versus dados originais<br><br>

Como não havia valores ausentes em nossos dados, é importante ter uma palavra sobre como lidar com valores ausentes com o OneHotEncoder. Um valor ausente pode ser facilmente tratado como um recurso extra. Observe que, para fazer isso, você precisa substituir primeiro o valor ausente por um valor arbitrário (por exemplo, 'missing'). pode apenas definir o parâmetro handle_unkown do OneHotEncoder para ignorar

#### a) Discretização<br>
A discretização, também conhecida como quantização ou binning, divide um recurso contínuo em um número pré-especificado de categorias (bins) e, assim, torna os dados discretos.<br>
Um dos principais objetivos de uma discretização é reduzir significativamente o número de intervalos discretos de um atributo contínuo. Por isso, por que essa transformação pode aumentar o desempenho de modelos baseados em árvore.<br>
O Sklearn fornece uma classe KBinsDiscretizer que pode cuidar disso. A única coisa que precisa especificar é o número de bins (n_bins) para cada recurso e como codificar esses bins (ordinal, onehot ou onehot-dense). O parâmetro de estratégia opcional pode ser definido com três valores:<br>
• uniforme, onde todas as caixas em cada recurso têm larguras idênticas.<br>
• quantil (padrão), em que todos os compartimentos em cada feature têm o mesmo número de pontos.<br>
• kmeans, onde todos os valores em cada bin têm o mesmo centro mais próximo de um cluster 1D k-means.<br>
É importante escolher o parâmetro de estratégia com cuidado. Usar a estratégia uniforme, por exemplo, é muito sensível para valores discrepantes e pode fazer com que acabe com caixas com apenas alguns pontos de dados, ou seja, os valores discrepantes.<br>
Vamos voltar ao nosso exemplo para alguns esclarecimentos. Importe a classe KBinsDiscretizer e crie uma nova instância com três compartimentos, codificação ordinal e uma estratégia uniforme (todos os compartimentos têm a mesma largura). Em seguida, ajuste e transforme todos os nossos indicadores originais e dados polinomiais ausentes.

In [33]:
from sklearn.preprocessing import KBinsDiscretizer

disc = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform') # encode='ordinal' instead of 'uniform'
disc.fit_transform(X1)

array([[2., 2., 1.],
       [0., 0., 2.],
       [1., 0., 0.],
       [1., 0., 1.]])

![](Picture%203.png)<br>
Discretized output<br>
Se a saída não fizer sentido, invoque o atributo bin_edges_ no discretizador (disco) e veja como os compartimentos são divididos. Em seguida, tente outra estratégia e veja como os limites do compartimento mudam de acordo.


#### b) Binarizacão<br>
A binarização de features é o processo de delimitar features numéricos para obter valores booleanos. Ou, por outras palavras, atribua um valor booleano (True ou False) a cada amostra com base em um limite. Observe que a binarização é uma forma extrema de discretização de dois compartimentos.<br>
Em geral, a binarização é útil como uma técnica de engenharia de features para criar novos features que indicam algo significativo. Assim como o MissingIndicator mencionado acima é usado para marcar valores ausentes significativos.<br>
A classe Binarizer no sklearn implementa a binarização de uma forma muito intuitiva. Os únicos parâmetros que precisa de especificar são o limite e a cópia. Todos os valores abaixo ou iguais ao limite são substituídos por 0, acima por 1. Se a cópia for definida como False, a binarização inplace será executada, caso contrário, será feita uma cópia.<br>
Considere o feature 3 (f3) do nosso exemplo e vamos criar um recurso binário extra com True para valores positivos e False para valores negativos. Importe a classe Binarizer, crie uma nova instância com o limite definido como zero e copie para True. Em seguida, ajuste e transforme o binarizador no feature 3. A saída é um novo array com valores booleanos.

In [16]:
from sklearn.preprocessing import Binarizer

binarizer = Binarizer(threshold=0, copy=True)
binarizer.fit_transform(X1.f3.values.reshape(-1, 1))
binarizer, X1

(Binarizer(threshold=0),
     f1   f2         f3
 0  5.0  7.0   8.000000
 1 -5.0  0.0  25.000000
 2  0.0  1.0  -1.000000
 3  0.0  0.0  10.666667)

#### c) Transformadores<br>
Se quer converter uma função existente em um transformador para auxiliar na limpeza ou processamento de dados, você pode implementar um transformador de uma função arbitrária com FunctionTransformer. Essa classe pode ser útil se estiver a trabalhar com um Pipeline no sklearn, mas pode ser facilmente substituída aplicando uma função lambda ao recurso que deseja transformar (como se mostra abaixo).

In [34]:
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log1p, validate=True)
transformer.fit_transform(X1.f2.values.reshape(-1, 1)) 
#same output
X1.f2.apply(lambda x : np.log1p(x)) #same output

0    2.079442
1    0.000000
2    0.693147
3    0.000000
Name: f2, dtype: float64

#### d) Dimensionamento de Features<br>
A próxima etapa lógica em nosso pipeline de pré-processamento é dimensionar as nossas features. Antes de aplicar qualquer transformação de dimensionamento, é muito importante dividir seus dados em um conjunto de treino e um conjunto de teste. Se começar a dimensionar antes, os seus dados de treino (e teste) podem ser dimensionados em torno de um valor médio (veja abaixo) que não é realmente a média dos dados de treino ou teste e ultrapassar todo o motivo pelo qual se está a dimensionar em primeiro lugar.

#### 6. Padronização<br>
A padronização é uma transformação que centraliza os dados removendo o valor médio de cada recurso e, em seguida, dimensiona-o dividindo recursos (não constantes) por seu desvio padrão. Após padronizar os dados, a média será zero e o desvio padrão um.<br>
A padronização pode melhorar drasticamente o desempenho dos modelos. Por exemplo, muitos elementos usados na função objetivo de um algoritmo de aprendizado (como o kernel RBF de Support Vector Machines ou os regularizadores l1 e l2 de modelos lineares) assumem que todos os recursos estão centrados em torno de zero e têm variância na mesma ordem. Se um recurso tem uma variância que é ordens de magnitude maior do que outros, ele pode dominar a função objetivo e tornar o estimador incapaz de aprender com outros recursos corretamente como esperado.
Dependendo de suas necessidades e dados, o sklearn fornece vários scalers: StandardScaler, MinMaxScaler, MaxAbsScaler e RobustScaler.

#### e) Escala padrão<br>
O principal scaler do Sklearn é o StandardScaler, usa uma definição estrita de padronização para padronizar dados. Ele centraliza puramente os dados usando a fórmula a seguir, em que u é a média e s é o desvio padrão.<br>
x_scaled = (x — u) / s<br>
Vamos apresentar exemplo para ver isso na prática. Antes de começarmos a codificar, devemos lembrar que o valor de nossa quarta instância estava faltando e o substituímos pela média. Se inserirmos a média na fórmula acima, o resultado após a padronização deve ser zero. Vamos testar isso.<br>
Importe a classe StandardScaler e crie uma nova instância. Observe que para matrizes esparsas você pode definir o parâmetro with_mean como False para não centralizar os valores em torno de zero. Em seguida, ajuste e transforme o scaler para o feature 3.

In [35]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit_transform(X1.f3.values.reshape(-1, 1))

array([[-0.28562322],
       [ 1.53522482],
       [-1.24960159],
       [ 0.        ]])

Como previsto, o valor da quarta feature é zero<br>
 ![](Picture%204.png)<br>
Ouput do standard scaling feature 3

#### f) Escala MinMax<br>
O MinMaxScaler transforma features, dimensionando cada recurso para um determinado intervalo. Esse intervalo pode ser definido especificando o parâmetro feature_range (padrão em (0,1)). Este scaler funciona melhor para casos em que a distribuição não é gaussiana ou o desvio padrão é muito pequeno. No entanto, é sensível a discrepâncias, portanto, se houver discrepâncias nos dados, convém considerar outro dimensionador.<br>
x_scaled = (x-min(x)) / (max(x)–min(x))<br>
Importar e usar o MinMaxScaler funciona — assim como todos os scalers a seguir — exatamente da mesma maneira que o StandardScaler. A única diferença está nos parâmetros na inicialização de uma nova instância.<br>
Aqui escalamos a feature 3 (f3) para uma escala entre -3 e 3. Como esperado, nosso valor máximo (25) é transformado em 3 e nosso valor mínimo (-1) é transformado em -3. Todos os outros valores são escalados linearmente entre esses valores.

In [19]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-3,3))
scaler.fit_transform(X1.f3.values.reshape(-1, 1))

array([[-0.92307692],
       [ 3.        ],
       [-3.        ],
       [-0.30769231]])

![](Picture%205.png)<br>
Feature 3 antes e depois de aplicado o MinMaxScaler

#### g) Escala MaxAbs<br>
O MaxAbsScaler funciona de maneira muito semelhante ao MinMaxScaler, mas dimensiona automaticamente os dados para um intervalo [-1,1] com base no máximo absoluto. Este scaler destina-se a dados que já estão centrados em zero ou dados esparsos. Ele não desloca/centraliza os dados e, portanto, não destrói nenhuma dispersão.<br>
x_scaled = x / max(abs(x))<br>
Vamos mais uma vez abordar o feature 3 transformando-o usando o MaxAbsScaler e comparando a saída com os dados originais.

In [36]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
scaler.fit_transform(X1.f3.values.reshape(-1, 1))

array([[ 0.32      ],
       [ 1.        ],
       [-0.04      ],
       [ 0.42666667]])

![](Picture%206.png)
<br>
Feature 3 antes e depois de aplicado o MaxAbsScaler

#### 7. Escala Robusta<br>
Se os seus dados contiverem muitos valores discrepantes, o dimensionamento usando a média e o desvio padrão dos dados provavelmente não funcionará muito bem. Nesses casos, pode usar o RobustScaler. Ele remove a mediana e dimensiona os dados de acordo com o intervalo de quantil. A fórmula exata do RobustScaler não é especificada pela documentação. Se quiser detalhes completos, pode sempre verificar o código-fonte.<br>
Por padrão, o scaler usa o Inter Quartile Range (IQR), que é o intervalo entre o 1º quartil e o 3º quartil. O intervalo de quantil pode ser definido manualmente especificando o parâmetro quantile_range ao iniciar uma nova instância do RobustScaler. Aqui, transformamos o feature 3 usando uma faixa de quantil de 10% a 90%.

In [37]:
from sklearn.preprocessing import RobustScaler

robust = RobustScaler(quantile_range = (0.1,0.9))
robust.fit_transform(X1.f3.values.reshape(-1, 1))

array([[ -6.17283951],
       [ 72.5308642 ],
       [-47.83950617],
       [  6.17283951]])

#### 8. Normalização<br>
A normalização é o processo de dimensionar amostras individuais para ter uma norma unitária. Em termos básicos, precisa normalizar os dados quando o algoritmo prevê com base nas relações ponderadas formadas entre os pontos de dados. Dimensionar entradas para normas de unidade é uma operação comum para classificação ou agrupamento de texto.<br>
Uma das principais diferenças entre o dimensionamento (por exemplo, padronização) e a normalização é que a normalização é uma operação de linha, enquanto o dimensionamento é uma operação de coluna.<br>
Embora existam muitas outras maneiras de normalizar os dados, o sklearn fornece três normas (o valor com o qual os valores individuais são comparados): l1, l2 e max.<br>
Ao criar uma nova instância da classe Normalizer, pode especificar a norma desejada no parâmetro norma.<br>
Abaixo, as fórmulas para as normas disponíveis são discutidas e implementadas em código Python — onde o resultado é uma lista de denominadores para cada amostra no conjunto de dados X .<br>
'máximo'<br>
 
A norma max usa o máximo absoluto e faz para amostras o que o MaxAbsScaler faz para recursos.<br>
x_normalized = x / max(x)

In [22]:
X1

,f1,f2,f3
0,5.0,7.0,8.000000
1,-5.0,0.0,25.000000
2,0.0,1.0,-1.000000
3,0.0,0.0,10.666667


In [23]:
norm_max = list(max(list(abs(i) for i in X1.iloc[r])) for r in range(len(X1)))
norm_max

[8.0, 25.0, 1.0, 10.666666666666666]

#### h) l1<br>
A norma l1 usa a soma de todos os valores como e, portanto, dá penalidades iguais a todos os parâmetros, reforçando a esparsidade.<br>
x_normalized = x / soma(X)

In [24]:
norm_l1 = list(sum(list(abs(i) for i in X1.iloc[r])) for r in range(len(X1)))
norm_l1

[20.0, 30.0, 2.0, 10.666666666666666]

#### i) l2<br>
A norma l2 usa a raiz quadrada da soma de todos os valores quadrados. Isso cria suavidade e invariância rotacional. Alguns modelos, como o PCA, assumem invariância rotacional e, portanto, l2 terá um desempenho melhor.<br>
x_normalized = x / sqrt(soma((i**2) for i in X))

In [25]:
import math
norm_l2 = list(math.sqrt(sum(list((i**2) for i in X1.iloc[r]))) for r in range(len(X1)))
norm_l2

[11.74734012447073, 25.495097567963924, 1.4142135623730951, 10.666666666666666]